In [3]:
import duckdb

# DuckDB veritabanını başlat
conn = duckdb.connect()

# Veriyi pickle dosyasından yükle
conn.execute("CREATE TABLE capstone_data AS SELECT * FROM read_csv('capstone_data.csv')")

# İlk 5 satırı göster
result = conn.execute("SELECT * FROM capstone_data LIMIT 5").fetchall()

# Sonuçları yazdır
for row in result:
    print(row)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

('51893f29-e6c2-45d3-807c-e1280d3d7b90', 18, 53.0, 'Prepaid', 106.74, 139.72, 47.31, 59.45, 0, None, 25, 18.0, 13, 1.34, '[]', False)
('a568caf1-d851-4847-a9f5-20ef9017fa92', 26, 15.0, 'Prepaid', 31.55, 12.14, 21.52, 1221.65, 0, None, 51, 8.0, 3, 2.5700000000000003, '[]', False)
('c611bf0e-a013-44dc-9939-bd33dab16d14', 32, 152.0, 'Postpaid', 30.64, 10.17, 31.11, 1170.45, 0, 1.0, 0, 13.0, 10, 8.02, '[]', False)
('9284b7b8-a4ef-49a7-9fa4-333954491f57', 33, 35.0, 'Prepaid', None, None, 36.03, 2418.91, 0, None, 19, 7.0, 11, 5.96, "['RitimGo']", False)
('e205b674-a6e6-43c5-aed7-9497c37c5c82', 18, 243.0, 'Prepaid', 85.62, 164.79, 46.4, 1518.19, 0, None, 99, 15.0, 6, 8.29, '[]', False)


In [4]:
# Tablo yapısını göster
result = conn.execute("DESCRIBE capstone_data").fetchall()

# Sonuçları yazdır
for row in result:
    print(row)

('id', 'VARCHAR', 'YES', None, None, None)
('age', 'BIGINT', 'YES', None, None, None)
('tenure', 'DOUBLE', 'YES', None, None, None)
('service_type', 'VARCHAR', 'YES', None, None, None)
('avg_call_duration', 'DOUBLE', 'YES', None, None, None)
('data_usage', 'DOUBLE', 'YES', None, None, None)
('roaming_usage', 'DOUBLE', 'YES', None, None, None)
('monthly_charge', 'DOUBLE', 'YES', None, None, None)
('overdue_payments', 'BIGINT', 'YES', None, None, None)
('auto_payment', 'DOUBLE', 'YES', None, None, None)
('avg_top_up_count', 'BIGINT', 'YES', None, None, None)
('call_drops', 'DOUBLE', 'YES', None, None, None)
('customer_support_calls', 'BIGINT', 'YES', None, None, None)
('satisfaction_score', 'DOUBLE', 'YES', None, None, None)
('apps', 'VARCHAR', 'YES', None, None, None)
('churn', 'BOOLEAN', 'YES', None, None, None)


In [5]:
# 'apps_tuple' adında yeni bir sütun ekle
conn.execute("""
    ALTER TABLE capstone_data ADD COLUMN apps_tuple VARCHAR
""")

# 'apps_tuple' sütununu güncelle
conn.execute("""
    UPDATE capstone_data
    SET apps_tuple = '(' || apps || ')'
""")

# Sonuçları kontrol et
result = conn.execute("SELECT apps, apps_tuple FROM capstone_data LIMIT 5").fetchall()

# Sonuçları yazdır
for row in result:
    print(row)

('[]', '([])')
('[]', '([])')
('[]', '([])')
("['RitimGo']", "(['RitimGo'])")
('[]', '([])')


In [6]:
# 'service_type' sütunundaki benzersiz değerleri al
service_type_unique = conn.execute("SELECT DISTINCT service_type FROM capstone_data").fetchall()

# 'apps' sütunundaki benzersiz değerleri al
apps_unique = conn.execute("SELECT DISTINCT apps FROM capstone_data").fetchall()

# Sonuçları yazdır
print("Unique service_type values:")
for row in service_type_unique:
    print(row)

print("\nUnique apps values:")
for row in apps_unique:
    print(row)

Unique service_type values:
('Broadband',)
('Postpaid',)
('Prepaid',)

Unique apps values:
("['RitimGo']",)
("['İzleGo', 'RitimGo', 'CüzdanX']",)
("['İzleGo', 'CüzdanX']",)
("['İzleGo', 'HızlıPazar']",)
("['Konuşalım']",)
("['İzleGo', 'Konuşalım']",)
("['RitimGo', 'HızlıPazar']",)
("['RitimGo', 'CüzdanX', 'HızlıPazar', 'Konuşalım']",)
("['İzleGo', 'RitimGo', 'CüzdanX', 'Konuşalım']",)
("['CüzdanX']",)
("['RitimGo', 'HızlıPazar', 'Konuşalım']",)
("['RitimGo', 'CüzdanX', 'Konuşalım']",)
("['RitimGo', 'Konuşalım']",)
("['RitimGo', 'CüzdanX', 'HızlıPazar']",)
("['İzleGo', 'CüzdanX', 'HızlıPazar', 'Konuşalım']",)
("['RitimGo', 'CüzdanX']",)
("['HızlıPazar', 'Konuşalım']",)
("['İzleGo', 'RitimGo']",)
("['HızlıPazar']",)
("['CüzdanX', 'Konuşalım']",)
("['CüzdanX', 'HızlıPazar']",)
("['İzleGo']",)
("['İzleGo', 'RitimGo', 'Konuşalım']",)
("['İzleGo', 'RitimGo', 'HızlıPazar']",)
("['CüzdanX', 'HızlıPazar', 'Konuşalım']",)
("['İzleGo', 'RitimGo', 'HızlıPazar', 'Konuşalım']",)
('[]',)
("['İzleGo',

In [7]:
# 'service_type' sütunundaki her benzersiz değeri ayrı sütunlara dönüştür
# Her benzersiz 'service_type' değeri için bir sütun oluşturuyoruz
service_types = conn.execute("SELECT DISTINCT service_type FROM capstone_data").fetchall()

# 'service_type' sütunundaki her benzersiz değeri 0 veya 1 olarak dönüştür
for service in service_types:
    service_value = service[0]
    column_name = f"service_type_{service_value.replace(' ', '_')}"
    
    # Yeni sütunu oluştur ve 1 veya 0 değeri atama
    conn.execute(f"""
        ALTER TABLE capstone_data ADD COLUMN {column_name} INT
    """)
    conn.execute(f"""
        UPDATE capstone_data
        SET {column_name} = CASE WHEN service_type = '{service_value}' THEN 1 ELSE 0 END
    """)

# Dönüştürülmüş verileri kontrol et
result = conn.execute("SELECT * FROM capstone_data LIMIT 5").fetchall()

# Sonuçları yazdır
for row in result:
    print(row)

('51893f29-e6c2-45d3-807c-e1280d3d7b90', 18, 53.0, 'Prepaid', 106.74, 139.72, 47.31, 59.45, 0, None, 25, 18.0, 13, 1.34, '[]', False, '([])', 1, 0, 0)
('a568caf1-d851-4847-a9f5-20ef9017fa92', 26, 15.0, 'Prepaid', 31.55, 12.14, 21.52, 1221.65, 0, None, 51, 8.0, 3, 2.5700000000000003, '[]', False, '([])', 1, 0, 0)
('c611bf0e-a013-44dc-9939-bd33dab16d14', 32, 152.0, 'Postpaid', 30.64, 10.17, 31.11, 1170.45, 0, 1.0, 0, 13.0, 10, 8.02, '[]', False, '([])', 0, 0, 1)
('9284b7b8-a4ef-49a7-9fa4-333954491f57', 33, 35.0, 'Prepaid', None, None, 36.03, 2418.91, 0, None, 19, 7.0, 11, 5.96, "['RitimGo']", False, "(['RitimGo'])", 1, 0, 0)
('e205b674-a6e6-43c5-aed7-9497c37c5c82', 18, 243.0, 'Prepaid', 85.62, 164.79, 46.4, 1518.19, 0, None, 99, 15.0, 6, 8.29, '[]', False, '([])', 1, 0, 0)


In [8]:
from prettytable import PrettyTable

# Sütun isimlerini al
columns = [col[1] for col in conn.execute("PRAGMA table_info(capstone_data)").fetchall()]

# Veriyi al
result = conn.execute("SELECT * FROM capstone_data LIMIT 5").fetchall()

# PrettyTable ile tabloyu oluştur
table = PrettyTable()
table.field_names = columns

# Sonuçları tablonun satırları olarak ekle
for row in result:
    table.add_row(row)

# Tabloyu yazdır
print(table)

+--------------------------------------+-----+--------+--------------+-------------------+------------+---------------+----------------+------------------+--------------+------------------+------------+------------------------+--------------------+-------------+-------+---------------+----------------------+------------------------+-----------------------+
|                  id                  | age | tenure | service_type | avg_call_duration | data_usage | roaming_usage | monthly_charge | overdue_payments | auto_payment | avg_top_up_count | call_drops | customer_support_calls | satisfaction_score |     apps    | churn |   apps_tuple  | service_type_Prepaid | service_type_Broadband | service_type_Postpaid |
+--------------------------------------+-----+--------+--------------+-------------------+------------+---------------+----------------+------------------+--------------+------------------+------------+------------------------+--------------------+-------------+-------+------------

In [9]:
# Benzersiz uygulama isimlerini çıkarma (DOĞRUDAN KELİMELER)
sql_query_unique_apps = """
    SELECT DISTINCT UNNEST(string_split(REPLACE(REPLACE(REPLACE(REPLACE(apps_tuple, '[', ''), ']', ''), '''', ''), ' ', ''), ','))
    FROM capstone_data
    WHERE apps_tuple IS NOT NULL AND apps_tuple <> '[]'
"""
apps_unique = [app[0].strip() for app in conn.execute(sql_query_unique_apps).fetchall() if app[0] and app[0].strip()]

# Mevcut sütunları al
existing_columns = [col[1] for col in conn.execute("PRAGMA table_info(capstone_data)").fetchall()]

# Yeni sütunlar oluşturma ve değerleri atama
for app in apps_unique:
    column_name = f"apps_tuple_{app.replace(' ', '_')}"

    # Sütun zaten var mı kontrol et (DAHA KESİN)
    if column_name not in existing_columns:
        try:
            # Sütunu ekle
            conn.execute(f"""
                ALTER TABLE capstone_data ADD COLUMN "{column_name}" BOOLEAN DEFAULT FALSE;
            """)
            print(f"Sütun oluşturuldu: {column_name}")  # Hangi sütunların oluşturulduğunu takip et
        except duckdb.CatalogException as e:  # CatalogException hatasını yakala
            print(f"Uyarı: Sütun oluşturulamadı (zaten var olabilir): {column_name} - {e}")
    else:
        print(f"Uyarı: Sütun zaten var: {column_name}")  # Hangi sütunların atlandığını takip et

    # Değerleri güncelle (DOĞRUDAN KELİMELER)
    conn.execute(f"""
        UPDATE capstone_data
        SET "{column_name}" = CASE WHEN apps_tuple LIKE ? THEN TRUE ELSE FALSE END
        """, ('%'+app+'%',))

# 'apps_tuple' sütununu silme (isteğe bağlı)
conn.execute("ALTER TABLE capstone_data DROP COLUMN apps_tuple")

# Sütun isimlerini al
columns = [col[1] for col in conn.execute("PRAGMA table_info(capstone_data)").fetchall()]

# Veriyi al
result = conn.execute("SELECT * FROM capstone_data LIMIT 5").fetchall()

# PrettyTable ile tabloyu oluştur
table = PrettyTable()
table.field_names = columns

# Sonuçları tablonun satırları olarak ekle
for row in result:
    table.add_row(row)

# Tabloyu yazdır
print(table)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Sütun oluşturuldu: apps_tuple_(Konuşalım)
Sütun oluşturuldu: apps_tuple_()
Sütun oluşturuldu: apps_tuple_(CüzdanX)
Sütun oluşturuldu: apps_tuple_HızlıPazar
Sütun oluşturuldu: apps_tuple_(RitimGo
Sütun oluşturuldu: apps_tuple_(CüzdanX
Sütun oluşturuldu: apps_tuple_RitimGo)
Sütun oluşturuldu: apps_tuple_(İzleGo
Sütun oluşturuldu: apps_tuple_(RitimGo)
Sütun oluşturuldu: apps_tuple_Konuşalım)
Sütun oluşturuldu: apps_tuple_CüzdanX)
Sütun oluşturuldu: apps_tuple_(HızlıPazar)
Sütun oluşturuldu: apps_tuple_RitimGo
Sütun oluşturuldu: apps_tuple_(HızlıPazar
Sütun oluşturuldu: apps_tuple_CüzdanX
Sütun oluşturuldu: apps_tuple_(İzleGo)
Sütun oluşturuldu: apps_tuple_HızlıPazar)
+--------------------------------------+-----+--------+--------------+-------------------+------------+---------------+----------------+------------------+--------------+------------------+------------+------------------------+--------------------+-------------+-------+----------------------+------------------------+---------

In [10]:
import re

# Sütun isimlerini al
columns = [col[1] for col in conn.execute("PRAGMA table_info(capstone_data)").fetchall()]

# Aynı isme sahip sütunları belirle
duplicate_columns = {}
for col in columns:
    if col.startswith("apps_tuple_"):
        # Sadece harf ve rakamlardan oluşan temel isim (DAHA TEMİZ)
        base_name = re.sub(r'[^a-zA-Z0-9]', '', col.replace("apps_tuple_", ""))
        if base_name not in duplicate_columns:
            duplicate_columns[base_name] = []
        duplicate_columns[base_name].append(col)

# Sütunları birleştir ve gereksiz sütunları sil
for base_name, cols in duplicate_columns.items():
    if len(cols) > 1:
        # Birleştirilmiş sütun için isim oluştur
        combined_column_name = f"apps_tuple_{base_name}"

        # Sütun zaten var mı kontrol et
        if combined_column_name not in columns:
            try:
                # Birleştirilmiş sütunu oluştur
                conn.execute(f"""
                    ALTER TABLE capstone_data ADD COLUMN "{combined_column_name}" BOOLEAN DEFAULT FALSE;
                """)
                print(f"Sütun oluşturuldu: {combined_column_name}")
            except duckdb.CatalogException as e:
                print(f"Uyarı: Sütun oluşturulamadı (zaten var olabilir): {combined_column_name} - {e}")
        else:
            print(f"Uyarı: Sütun zaten var: {combined_column_name}")

        # Değerleri güncelle
        conn.execute(f"""
            UPDATE capstone_data
            SET "{combined_column_name}" = {' OR '.join([f'"{col}"' for col in cols])}
        """)

        # Gereksiz sütunları sil
        for col in cols:
            conn.execute(f"""
                ALTER TABLE capstone_data DROP COLUMN "{col}";
            """)
        print(f"Sütunlar silindi: {cols} -> {combined_column_name}")
    else:
        print(f"Tek sütun bulundu, birleştirmeye gerek yok: {cols[0]}")

# Sütun isimlerini al
columns = [col[1] for col in conn.execute("PRAGMA table_info(capstone_data)").fetchall()]

# Veriyi al
result = conn.execute("SELECT * FROM capstone_data LIMIT 5").fetchall()

# PrettyTable ile tabloyu oluştur
table = PrettyTable()
table.field_names = columns

# Sonuçları tablonun satırları olarak ekle
for row in result:
    table.add_row(row)

# Tabloyu yazdır
print(table)

Sütun oluşturuldu: apps_tuple_Konualm
Sütunlar silindi: ['apps_tuple_(Konuşalım)', 'apps_tuple_Konuşalım)'] -> apps_tuple_Konualm
Tek sütun bulundu, birleştirmeye gerek yok: apps_tuple_()
Sütun oluşturuldu: apps_tuple_CzdanX
Sütunlar silindi: ['apps_tuple_(CüzdanX)', 'apps_tuple_(CüzdanX', 'apps_tuple_CüzdanX)', 'apps_tuple_CüzdanX'] -> apps_tuple_CzdanX
Sütun oluşturuldu: apps_tuple_HzlPazar
Sütunlar silindi: ['apps_tuple_HızlıPazar', 'apps_tuple_(HızlıPazar)', 'apps_tuple_(HızlıPazar', 'apps_tuple_HızlıPazar)'] -> apps_tuple_HzlPazar
Uyarı: Sütun zaten var: apps_tuple_RitimGo
Sütunlar silindi: ['apps_tuple_(RitimGo', 'apps_tuple_RitimGo)', 'apps_tuple_(RitimGo)', 'apps_tuple_RitimGo'] -> apps_tuple_RitimGo
Sütun oluşturuldu: apps_tuple_zleGo
Sütunlar silindi: ['apps_tuple_(İzleGo', 'apps_tuple_(İzleGo)'] -> apps_tuple_zleGo
+--------------------------------------+-----+--------+--------------+-------------------+------------+---------------+----------------+------------------+-------

In [11]:
# Yeniden adlandırılacak sütunlar
column_renames = {
    "apps_tuple_CzdanX": "apps_tuple_CuzdanX",
    "apps_tuple_Konualm": "apps_tuple_Konusalim",
    "apps_tuple_zleGo": "apps_tuple_IzleGo",
    "apps_tuple_HzlPazar": "apps_tuple_HizliPazar"
}

# Sütunları yeniden adlandırma
for old_name, new_name in column_renames.items():
    try:
        conn.execute(f"""
            ALTER TABLE capstone_data RENAME COLUMN "{old_name}" TO "{new_name}";
        """)
        print(f"Sütun yeniden adlandırıldı: {old_name} -> {new_name}")
    except Exception as e:
        print(f"Hata: Sütun yeniden adlandırılamadı: {old_name} - {e}")

# Sütun isimlerini al
columns = [col[1] for col in conn.execute("PRAGMA table_info(capstone_data)").fetchall()]

# Veriyi al
result = conn.execute("SELECT * FROM capstone_data LIMIT 5").fetchall()

# PrettyTable ile tabloyu oluştur
table = PrettyTable()
table.field_names = columns

# Sonuçları tablonun satırları olarak ekle
for row in result:
    table.add_row(row)

# Tabloyu yazdır
print(table)

Sütun yeniden adlandırıldı: apps_tuple_CzdanX -> apps_tuple_CuzdanX
Sütun yeniden adlandırıldı: apps_tuple_Konualm -> apps_tuple_Konusalim
Sütun yeniden adlandırıldı: apps_tuple_zleGo -> apps_tuple_IzleGo
Sütun yeniden adlandırıldı: apps_tuple_HzlPazar -> apps_tuple_HizliPazar
+--------------------------------------+-----+--------+--------------+-------------------+------------+---------------+----------------+------------------+--------------+------------------+------------+------------------------+--------------------+-------------+-------+----------------------+------------------------+-----------------------+---------------+----------------------+--------------------+-----------------------+-------------------+
|                  id                  | age | tenure | service_type | avg_call_duration | data_usage | roaming_usage | monthly_charge | overdue_payments | auto_payment | avg_top_up_count | call_drops | customer_support_calls | satisfaction_score |     apps    | churn | serv

In [12]:
# Boolean sütunları belirleme
boolean_columns = []
pragma_result = conn.execute("PRAGMA table_info(capstone_data)").fetchall()
for col in pragma_result:
    if col[2] == "BOOLEAN":
        boolean_columns.append(col[1])

# Boolean sütunları 0 ve 1'e dönüştürme
for col in boolean_columns:
    try:
        # Değerleri güncelleme
        conn.execute(f"""
            UPDATE capstone_data
            SET "{col}" = CASE WHEN "{col}" = TRUE THEN 1 WHEN "{col}" = FALSE THEN 0 ELSE NULL END;
        """)

        # Sütun tipini değiştirme
        conn.execute(f"""
            ALTER TABLE capstone_data ALTER COLUMN "{col}" TYPE INTEGER;
        """)
        print(f"Sütun dönüştürüldü: {col}")
    except Exception as e:
        print(f"Hata: Sütun dönüştürülemedi: {col} - {e}")

# Sütun isimlerini al
columns = [col[1] for col in conn.execute("PRAGMA table_info(capstone_data)").fetchall()]

# Veriyi al
result = conn.execute("SELECT * FROM capstone_data LIMIT 5").fetchall()

# PrettyTable ile tabloyu oluştur
table = PrettyTable()
table.field_names = columns

# Sonuçları tablonun satırları olarak ekle
for row in result:
    table.add_row(row)

# Tabloyu yazdır
print(table)

Sütun dönüştürüldü: churn
Sütun dönüştürüldü: apps_tuple_()
Sütun dönüştürüldü: apps_tuple_Konusalim
Sütun dönüştürüldü: apps_tuple_CuzdanX
Sütun dönüştürüldü: apps_tuple_HizliPazar
Sütun dönüştürüldü: apps_tuple_IzleGo
+--------------------------------------+-----+--------+--------------+-------------------+------------+---------------+----------------+------------------+--------------+------------------+------------+------------------------+--------------------+-------------+-------+----------------------+------------------------+-----------------------+---------------+----------------------+--------------------+-----------------------+-------------------+
|                  id                  | age | tenure | service_type | avg_call_duration | data_usage | roaming_usage | monthly_charge | overdue_payments | auto_payment | avg_top_up_count | call_drops | customer_support_calls | satisfaction_score |     apps    | churn | service_type_Prepaid | service_type_Broadband | service_type_P

In [13]:
# Sütun isimleri (sizin sağladığınız)
columns = [
    "id", "age", "tenure", "service_type", "avg_call_duration", "data_usage",
    "roaming_usage", "monthly_charge", "overdue_payments", "auto_payment",
    "avg_top_up_count", "call_drops", "customer_support_calls",
    "satisfaction_score", "apps", "churn", "service_type_Postpaid",
    "service_type_Prepaid", "service_type_Broadband", "apps_tuple_()",
    "apps_tuple_CuzdanX", "apps_tuple_Konusalim", "apps_tuple_IzleGo",
    "apps_tuple_HizliPazar"
]

# Sorguyu dinamik olarak oluştur
select_expressions = []
for col in columns:
    select_expressions.append(f"COUNT(CASE WHEN \"{col}\" IS NULL THEN 1 END) AS \"{col}\"")

sql_query = f"""
    SELECT {', '.join(select_expressions)}
    FROM capstone_data
"""

# Sorguyu çalıştırma ve sonucu alma
result = conn.execute(sql_query).fetchdf()

# Sonucu yazdırma
print(result)

   id  age  tenure  service_type  avg_call_duration  data_usage  \
0   0    0   24800             0            3664313      499533   

   roaming_usage  monthly_charge  overdue_payments  auto_payment  ...  apps  \
0        3331212          250071                 0       3344748  ...     0   

   churn  service_type_Postpaid  service_type_Prepaid  service_type_Broadband  \
0      0                      0                     0                       0   

   apps_tuple_()  apps_tuple_CuzdanX  apps_tuple_Konusalim  apps_tuple_IzleGo  \
0              0                   0                     0                  0   

   apps_tuple_HizliPazar  
0                      0  

[1 rows x 24 columns]


In [15]:
import pandas as pd
import lightgbm as lgb

# Veritabanından veriyi DataFrame'e çekme
capstone_data_df = conn.execute("""
    SELECT * FROM capstone_data;
""").fetchdf()

# Eksik verilerin olduğu sütunları tespit edelim
missing_columns = capstone_data_df.columns[capstone_data_df.isnull().any()].tolist()
print("Eksik degerler olan sutunlar:", missing_columns)

# Eksik veriyi doldurmak için modelleme fonksiyonu
def fill_missing_values_with_model(df, target_column):
    # Eksik olmayan verilerle modeli eğitmek için X ve y'yi ayır
    train_data = df.dropna(subset=[target_column])
    
    # 'target_column' dışındaki sütunları seç, ancak gereksiz sütunları dışla
    X_train = train_data.drop(columns=[target_column, 'id', 'service_type', 'apps', 'apps_tuple_()'])
    
    # Bu adımda yalnızca 'id', 'service_type' ve 'apps' gibi gereksiz sütunların olmadığından emin olun
    print("X_train sütunları:", X_train.columns.tolist())
    
    # Hedef değişkeni ayır
    y_train = train_data[target_column]

    # Modeli oluştur ve eğit
    model = lgb.LGBMRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    model.fit(X_train, y_train)

    # Eksik verileri tahmin et
    missing_rows = df[df[target_column].isnull()]
    X_missing = missing_rows.drop(columns=[target_column, 'id', 'service_type', 'apps', 'apps_tuple_()'])
    
    predicted_values = model.predict(X_missing)

    # Tahmin edilen değerleri yerine koy
    df.loc[missing_rows.index, target_column] = predicted_values

    return df

# Eksik sütunlar üzerinde model eğitimi ve doldurma işlemi
for column in missing_columns:
    capstone_data_df = fill_missing_values_with_model(capstone_data_df, column)

# Güncellenmiş veriyi CSV olarak kaydet
capstone_data_df.to_csv("capstone_data_filled.csv", index=False)

# Değişiklikleri göster
print("Eksik değerler dolduruldu!")
print(capstone_data_df.isnull().sum())  # Eksik değer kontrolü

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Eksik degerler olan sutunlar: ['tenure', 'avg_call_duration', 'data_usage', 'roaming_usage', 'monthly_charge', 'auto_payment', 'call_drops']
X_train sütunları: ['age', 'avg_call_duration', 'data_usage', 'roaming_usage', 'monthly_charge', 'overdue_payments', 'auto_payment', 'avg_top_up_count', 'call_drops', 'customer_support_calls', 'satisfaction_score', 'churn', 'service_type_Prepaid', 'service_type_Broadband', 'service_type_Postpaid', 'apps_tuple_Konusalim', 'apps_tuple_CuzdanX', 'apps_tuple_HizliPazar', 'apps_tuple_IzleGo']
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.232222 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1500
[LightGBM] [Info] Number of data points in the train set: 9975200, number of used features: 17
[LightGBM] [Info] Start training from score 158.251774
X_train sütunları: ['age', 'tenure', 'data_usage', 'roamin

In [17]:
capstone_data_filled_df = pd.read_csv("capstone_data_filled.csv")
capstone_data_filled_df.head()

,id,age,tenure,service_type,avg_call_duration,data_usage,roaming_usage,monthly_charge,overdue_payments,auto_payment,...,apps,churn,service_type_Prepaid,service_type_Broadband,service_type_Postpaid,apps_tuple_(),apps_tuple_Konusalim,apps_tuple_CuzdanX,apps_tuple_HizliPazar,apps_tuple_IzleGo
0,51893f29-e6c2-45d3-807c-e1280d3d7b90,18,53.0,Prepaid,106.740000,139.720000,47.31,59.45,0,0.500842,...,[],0,1,0,0,0,0,0,0,0
1,a568caf1-d851-4847-a9f5-20ef9017fa92,26,15.0,Prepaid,31.550000,12.140000,21.52,1221.65,0,0.501090,...,[],0,1,0,0,0,0,0,0,0
2,c611bf0e-a013-44dc-9939-bd33dab16d14,32,152.0,Postpaid,30.640000,10.170000,31.11,1170.45,0,1.000000,...,[],0,0,0,1,0,0,0,0,0
3,9284b7b8-a4ef-49a7-9fa4-333954491f57,33,35.0,Prepaid,60.411314,99.841751,36.03,2418.91,0,0.509482,...,['RitimGo'],0,1,0,0,0,0,0,0,0
4,e205b674-a6e6-43c5-aed7-9497c37c5c82,18,243.0,Prepaid,85.620000,164.790000,46.40,1518.19,0,0.500434,...,[],0,1,0,0,0,0,0,0,0


In [18]:
capstone_data_df = pd.read_csv("capstone_data.csv")
capstone_data_df.head()

,id,age,tenure,service_type,avg_call_duration,data_usage,roaming_usage,monthly_charge,overdue_payments,auto_payment,avg_top_up_count,call_drops,customer_support_calls,satisfaction_score,apps,churn
0,51893f29-e6c2-45d3-807c-e1280d3d7b90,18,53.0,Prepaid,106.74,139.72,47.31,59.45,0,NaN,25,18.0,13,1.34,[],False
1,a568caf1-d851-4847-a9f5-20ef9017fa92,26,15.0,Prepaid,31.55,12.14,21.52,1221.65,0,NaN,51,8.0,3,2.57,[],False
2,c611bf0e-a013-44dc-9939-bd33dab16d14,32,152.0,Postpaid,30.64,10.17,31.11,1170.45,0,1.0,0,13.0,10,8.02,[],False
3,9284b7b8-a4ef-49a7-9fa4-333954491f57,33,35.0,Prepaid,NaN,NaN,36.03,2418.91,0,NaN,19,7.0,11,5.96,['RitimGo'],False
4,e205b674-a6e6-43c5-aed7-9497c37c5c82,18,243.0,Prepaid,85.62,164.79,46.40,1518.19,0,NaN,99,15.0,6,8.29,[],False


In [23]:
capstone_data_filled_df["auto_payment"] = (capstone_data_filled_df["auto_payment"] >= 0.6).astype(int)

In [27]:
capstone_data_filled_df.to_csv("capstone_data_filled_new.csv", index=False)

In [29]:
capstone_data_filled_new_df = pd.read_csv("capstone_data_filled_new.csv")
capstone_data_filled_new_df["call_drops"].head()

0    18.0
1     8.0
2    13.0
3     7.0
4    15.0
Name: call_drops, dtype: float64

In [30]:
conn.close()

In [ ]:
# auc
# sim kart atma churn, 30 gün boyunca aktivitesi yoksa
# post paid, gidince kaybın ne kadar olacagı hesaplanır, en sevilen müşteri
# kalabalık, gürültü vs de pre paid den çıkarılır
